# Option pricing

The goal here is to do an option pricing exercise. Inspired by [this blog post](http://firsttimeprogrammer.blogspot.co.uk/2015/08/european-option-pricing-with-python.html).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from math import sqrt, exp
import numpy.random as rn

## Definitions and concepts

### Geometric Brownian motion

Without going into a derivation, we can write
$$ S(t_i) = S_0 \prod_{k=1}^i Y_k $$
where
$$ Y_i = Y(t_i) = \exp\left( \sigma\sqrt{t_i - t_{i-1}}Z_i + \mu(t_i - t_{i-1}) \right) $$
and the $Z_i \sim N(0,1)$ are all *i.i.d.*

In [ ]:
def gbrownian(t, S0, mu, sigma):
    dt = np.diff(t)
    Z = rn.normal(size=dt.size)
    S = np.array([exp(sigma*sqrt(dti)*Zi + mu*dti) for dti, Zi in zip(dt, Z)])
    S = np.insert(S, 0, S0)
    return np.cumprod(S)

### Stock price drift and variance

Define a *return* $r_i$ at time $t_i$ given prices $S_i$ and $S_{i-1}$ where $t_i - t_{i-1} > 0$ as
$$ r_i = \frac{p_i - S_{i-1}}{S_{i-1}}. $$
This measures *percent* change. Then $\log(1 + r_i)$ is normally distributed assuming prices are in a log normal distribution since
$$ 1 + r_i = \frac{S_i}{S_{i-1}} = e^{\log\left(\frac{S_i}{S_{i-1}}\right)}. $$

Drift ($\mu$) and volatility ($\sigma$) are defined in terms of expected value ($U$) and variance ($V^2$) of the log of price ratios. Let $n$ be the number of trading days (number of price points), which means $\Delta t = 1/n$. Then
$$ U = \frac{1}{n-1} \sum_{i=1}^{n-1} \log\left( \frac{S_{i+1}}{S_i} \right) $$
and
$$ V^2 = \frac{1}{n-2} \sum_{i=1}^{n-1} \left( \log\left( \frac{S_{i+1}}{S_i} \right) - U \right)^2. $$
Note the last is a *sample* variance. Volatility and drift are then
$$ \mu = \frac{U + V^2/2}{\Delta t} $$
and
$$ \sigma = \frac{V}{\sqrt{\Delta t}}. $$

#### Example

* We'll use IBM stock prices to compute drift and volatility. Prices downloaded from Google.

In [ ]:
ibm = pd.read_csv('ibm.csv')
S = ibm[' Close']
n = S.size

In [ ]:
lograt = np.diff(np.log(S))
U = np.sum(lograt)/(n - 1)
V2 = np.sum((lograt - U)**2)/(n - 2)
dt = 1/n
mu = (U + V2/2)/dt
sigma = sqrt(V2/dt)
print('Drift:', mu)
print('Volatility:', sigma)

* Now we'll consider 10 simulations of price using the calculated drift and volatility, and plot these (in chartreuse) against the actual price (in black).

In [ ]:
t = np.arange(0, n, dtype=float)/(n - 1)
S0 = S[0]

for _ in range(0, 10):
    Sb = gbrownian(t, S0, mu, sigma)
    plt.plot(Sb, 'xkcd:chartreuse')

plt.plot(S, 'k')
plt.xlim([0, n-1])
plt.show()

## European option price

In [ ]:
S0 = 173.39
strike = 130.
rr = 0.03     # Risk free rate. WAG, because I don't know any better.
Nmc = 100000   # Number of trials.
days = 2     # Number of trading days to consider.
nopay = 0
payoffs = []

for i in range(Nmc):
    sim = gbrownian(np.arange(0, days)/(days - 1), S0, mu, sigma)
    sim = sim[-1]
    if sim > strike:
        payoffs.append((sim - strike)*exp(-rr/365.*days))
    else:
        nopay += 1

payoffs = np.array(payoffs)
price = np.sum(payoffs)/payoffs.size

print('Price: ', price)
print('No pay ratio: ', nopay/float(Nmc))

# References

* [Why log returns](https://quantivity.wordpress.com/2011/02/21/why-log-returns/)
* [Drift and volatility of market prices](http://www.lifelong-learners.com/opt/SYL/s1node6.php)
* [Texas A&M maths answer sheet](http://www.math.tamu.edu/~stecher/425/Sp01/exam2.pdf)